## **Advances in Data Mining**

Stephan van der Putten | (s1528459) | stvdputtenjur@gmail.com  
Theo Baart | s2370328 | s2370328@student.leidenuniv.nl

### **Assignment 3**
This assignment is concerned with performing an analysis of and execute PageRank on the wikipedia links given in the `wikilink_graph.2004-03-01.csv` file. In order to do this the assignment is split up into four subtasks with each subtask receiving its dedicated `.ipynb` file. See each specific file for details on what this notebook accomplishes.

Note all implementations are based on the assignment guidelines and helper files given as well as the documentation of the used functions. Additionally, the following sources are referenced:
  1. Mining of Masive Datasets by Jure Leskovec, Anand Rajaraman and Jeff ullman

#### **Exploratory Data Analysis**
This notebook performs an exploratory analysis on the dataset. This includes some anlaysis on the nodes and edges as well as estimating system requirements for being able to execute the PageRank algorithm.
___

### **Helper Functions**
This section contains functions which aid and simplify the code.
___
The following snippet handles all imports.

In [19]:
%matplotlib inline
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.sparse
from scipy.sparse import csr_matrix

The `compute_average` function computes the average value of a one-dimensional sparse matrix.

In order to do this the function uses the following parameters:
  * `data` - the one-dimensional sparse matrix we want the average of
  * `ignore_zero` - whether to ignore zeros [default = `True`]

Additionally, it returns the following value:
  * `average` - the average value of the given matrix.

In [2]:
def compute_average(data,ignore_zero=True):
    if ignore_zero:
        # built in functions don't ignore zeros
        d_sum = data.sum()
        d_len = data.count_nonzero()
        average = d_sum / d_len
    else:
        average = data.mean()
    return average

The `compute_average_link_count` function parses the nodes and computes the average number of links per node.

In order to do this the function uses the following parameter:
  * `data` - the prepped data
  
Additionally, it returns the following value:
  * `average` - the average number of links per node

In [3]:
def compute_average_link_count(data):
    num_nodes = data.get_shape()[0]
    num_links = data.getnnz()
    average = num_nodes / num_links
    return average

The `compute_dead_ends_set` function parses the nodes and computes a set of all nodes which are dead_ends. Analyzing the data it is evident that for a node to be in the dataset it must either have an outgoing edge or an incoming edge. By definition a dead end has no outgoing edges and therefore it cannot be in the list of outgoing edges. Thus, the difference between the set of all edges and the set of outgoing edges is the set of dead ends.

In order to do this the function uses the following parameter:
  * `data` - the prepped data
  
Additionally, it returns the following value:
  * `dead_ends` - a list of all the dead ends [in consecutive numbering]

In [4]:
def compute_dead_ends_set(data):
    all_edges = set(range(data.get_shape()[1])) # get all column ID's 
    outgoing_edges = set(data.nonzero()[1]) # column ID's of outgoing edges
#     print(all_edges)
#     print(outgoing_edges)
    dead_ends = all_edges - outgoing_edges
#     print(dead_ends)
    dead_ends = set(dead_ends)
    return dead_ends

The `compute_ram_custom_format` function analyzes the ram requirements using the custom format specified on slide 17 of the instructional slideset. The RAM computation is based on the description given in Section 5.2.1 of [1].

In order to do this the function uses the following parameters:
  * `num_links` - how many links there are in the dataset
  * `byte_format` - in what unit RAM should be returned [supports: `[GB, MB, KB, B, compute]`, default = `compute`]
  * `bits` - how many bits are needed to store a value [default = `32` as we only need to store the row numbers and there are less than 4 million rows]
  * `avg_links` - how many destinations there are on average per node [default = `10` based on slide 16 in the instructional slideset]
  
Additionally, it returns the following value:
  * `total_bytes` - the number of bytes needed in RAM
  * `unit_bytes` - the byte format used

In [5]:
def compute_ram_custom_format(num_links, byte_format = 'compute', bits = 32, avg_links = 10):
    # transition_matrix contains num_links elements with avg_links + 1 (degree) values
    bits_transition_matrix = (num_links * (avg_links + 1)) * bits
    # initial vector contains num_links values.
    bits_initial_vector = (num_links) * bits
    total_bits = bits_transition_matrix + bits_initial_vector
    # convert to Xbytes
    total_bytes = total_bits / 8
    if byte_format == 'compute':
        total_bytes, unit_bytes = compute_x_bytes(total_bytes)
    else:
        total_bytes, unit_bytes = compute_x_bytes(total_bytes,byte_format)        
    return total_bytes, unit_bytes   

The `compute_ram_sparse` function analyzes the ram requirements using a sparce matrix as the implementation of the transition matrix and the initial vector. The RAM computation is based on the description given in Section 5.2.1 of [1].

In order to do this the function uses the following parameters:
  * `num_links` - how many links there are in the dataset
  * `byte_format` - in what unit RAM should be returned [supports: `[GB, MB, KB, B, compute]`, default = `compute`]
  * `bits` - how many bits are needed to store a value [default = `64`]
  * `avg_links` - how many destinations there are on average per node [default = `10` based on slide 16 in the instructional slideset]
  
Additionally, it returns the following value:
  * `total_bytes` - the number of bytes needed in RAM
  * `unit_bytes` - the byte format used

In [6]:
def compute_ram_sparse(num_links, byte_format = 'compute', bits = 64, avg_links = 10):
    # transition_matrix contains num_links elements with avg_links values. Additionally, values and coordinates must be stored.
    value_bits = (num_links * avg_links) * bits
    coordinate_bits = (num_links * avg_links) * 64 # coordinates are 32bits, 2 coordinates per value
    bits_transition_matrix = value_bits + coordinate_bits
    # initial vector contains num_links values.
    bits_initial_vector = (num_links) * bits
    total_bits = bits_transition_matrix + bits_initial_vector
    # convert to Xbytes
    total_bytes = total_bits / 8
    if byte_format == 'compute':
        total_bytes, unit_bytes = compute_x_bytes(total_bytes)
    else:
        total_bytes, unit_bytes = compute_x_bytes(total_bytes,byte_format)        
    return total_bytes, unit_bytes     

The `compute_ram_traditional` function analyzes the ram requirements using the traditional implementation of the transition matrix and the initial vector. Note that double precision is assumed (e.g. 64 bits per value)

In order to do this the function uses the following parameters:
  * `num_links` - how many links there are in the dataset
  * `byte_format` - in what unit RAM should be returned [supports: `[GB, MB, KB, B, compute]`, default = `compute`]
  * `bits` - how many bits are needed to store a value [default = `64`]
  
Additionally, it returns the following value:
  * `total_bytes` - the number of bytes needed in RAM
  * `unit_bytes` - the byte format used

In [7]:
def compute_ram_traditional(num_links, byte_format = 'compute', bits = 64):
    # transition_matrix contains num_links * num_links values
    bits_transition_matrix = (num_links * num_links) * bits
    # initial vector contains num_links values.
    bits_initial_vector = (num_links) * bits
    total_bits = bits_transition_matrix + bits_initial_vector
    # convert to Xbytes
    total_bytes = total_bits / 8
    if byte_format == 'compute':
        total_bytes, unit_bytes = compute_x_bytes(total_bytes)
    else:
        total_bytes, unit_bytes = compute_x_bytes(total_bytes,byte_format)        
    return total_bytes, unit_bytes   

The `compute_x_bytes` function receives a value and converts it to the closest logical unit. This is defined by being the largest unit such that there is always a value before the decimal point. The largest allowed unit is `GB` and additionally the value is always rounded to 3 decimal places.

In order to do this the function uses the following parameters:
  * `total_bytes` - the value to convert
  * `format_override` - use to manually force a format [supports: `[GB, MB, KB, B, None]`, default = `None`]
  
Additionally, it returns the following value:
  * `converted_bytes` - the converted value rounded to 3 decimal places
  * `unit_bytes` - a textual representation of the unit used

In [8]:
def compute_x_bytes(total_bytes, format_override=None):
    if format_override is not None:
        unit_bytes = format_override
        if format_override == 'B':
            converted_bytes = total_bytes
        elif format_override == 'KB':
            converted_bytes = total_bytes / 1e3
        elif format_override == 'MB':
            converted_bytes = total_bytes / 1e6
        elif format_override == 'GB':
            converted_bytes = total_bytes / 1e9
    else :
        if total_bytes % 1e9 < total_bytes:
            converted_bytes = total_bytes / 1e9
            unit_bytes = 'GB'
        elif total_bytes % 1e6 < total_bytes:
            converted_bytes = total_bytes / 1e6
            unit_bytes = 'MB'
        elif total_bytes % 1e3 < total_bytes:
            converted_bytes = total_bytes / 1e3
            unit_bytes = 'KB'
        else:
            converted_bytes = total_bytes
            unit_bytes = 'B'
    converted_bytes = round(converted_bytes,3)
    return converted_bytes, unit_bytes   

The `compute_x_degrees` function parses the nodes and computes for each node the x-degree of the node.

In order to do this the function uses the following parameters:
  * `data` - the prepped data
  * `x` - whether to compute the in- or out-degree [default = `in`]
  
Additionally, it returns the following value:
  * `x_degrees` - a list of all the nodes and their x-degrees

In [9]:
def compute_x_degrees(data, x = 'in'):
    if x == 'in':
        edges = data.nonzero()[0]
        edges_length = data.get_shape()[0]
    else:
        edges = data.nonzero()[1]
        edges_length = data.get_shape()[1]
    edges_index, edges_count = np.unique(edges, return_counts=True)
    edges_nonzero_length = len(edges_index)
    edges_zeros = np.zeros(edges_nonzero_length)
#     print(edges_count)
#     print(np.log(edges_count))
#     print(edges_index,edges_count,edges_zeros,edges_nonzero_length,edges_length)
    x_degrees = csr_matrix((edges_count, (edges_zeros,edges_index)), shape=(1,edges_length))
#     print(x_degrees)
    return x_degrees

The `plot_distribution` function plots the distribution of a one-dimensional sparse matrix.

In order to do this the function uses the following parameters:
  * `data` - the one-dimensional sparse matrix to be plotted
  * `title` - the title to give the plot [default = `'distribution'`]
  * `ignore_zeros` - whether to include zeros in the distrubtion [default = `True`]

In [139]:
def plot_distribution(data,title='Count of each Degree ',ignore_zeros=True):
    #TODO Implement Log optionality
     
    data = data.toarray()[0]
    if ignore_zeros:
        data = data[data!=0]
    degree, count = np.unique(data, return_counts=True)

#     print(np.unique(data, return_counts=True))
    #plotting
    fig, ax = plt.subplots()
    plt.plot(degree, count)
    plt.title(title)
    ax.set_xlabel('log degree')
    ax.set_ylabel('log count')
    plt.yscale('log')
    plt.xscale('log')
#     plt.savefig('graph_in.png', dpi=100)
    plt.show()

### **ExploratoryDataAnalysis Class**

This section contains the class and functions responsible for the exploratory data analysis. Due to the limitations of `.ipynb` files the various functions will be described first and then the implementation will be shown.

___
The `__init__` function initializes the class.

In order to do this the function uses the following (optional) parameter:
  * `prepped_dataset` - the prepared link data [default: None]
___
The `load_prepped_data` function is responsible for retrieving the data prepped by `prep.ipynb` and loading it for exploratory data analysis.

In order to do this the function uses the following parameter:
  * `filename` - the name of the file containing the prepped data [default = `prep_data.npz`]
___
The `analyse_dead_ends` function analyzes the matrix and prints some data on the dead ends in the graph. A dead end refers to nodes which do not have any outgoing edges.

In order to do this the function uses the following parameter:
  * `print_set`  - whether to print the set of dead ends [default = `False`]
___
The `analyze_in_degrees` function analyzes the matrix and prints some data on the in-degrees of the graph. An in-degree refers to the number of incoming edges that a node has. This includes a distribution of the in-degrees as well as the average number of in-degrees.

In order to do this the function uses the following parameter:
  * `ignore_zeros_plot` - whether to plot zeros [default = `True`]
___
The `analyze_out_degrees` function analyzes the matrix and prints some data on the out-degrees of the graph. An out-degree refers to the number of outgoinng edges that a node has. This includes a distribution of the out-degrees as well as the average number of out-degrees.

In order to do this the function uses the following parameter:
  * `ignore_zeros_plot` - whether to plot zeros [default = `True`]
___
The `analyze_ram_requirements` function analyzes the transition matrix and various methods of implementing it in order to estimate the ram requirements needed. 

In [26]:
class ExploratoryDataAnalysis():
    def __init__(self,prepped_dataset=None):
        self.prepped_dataset = prepped_dataset
    
    def load_prepped_data(self,filename = 'prep_data.npz'):
        self.prepped_dataset = scipy.sparse.load_npz(filename)
    
    def analyze_dead_ends(self,print_set=False):
        dead_ends = compute_dead_ends_set(self.prepped_dataset)
        count_dead_ends = len(dead_ends)
        if count_dead_ends == 0:
            print("There are no dead ends")
            return
        # TODO convert consecutive numbering to original number
        if count_dead_ends == 1:
            print("There is 1 dead end.")
        else:
            print("There are "+str(len(dead_ends))+" dead ends.")
        if print_set:
            print("The following set contains all nodes classified as dead ends [in consecutive numbering].")
            print(dead_ends)
            
    def analyze_in_degrees(self,ignore_zeros_plot = True):
        in_degrees = compute_x_degrees(self.prepped_dataset, 'in')

        # Distribution
        plot_distribution(in_degrees,title='Log distribution of in-degrees',ignore_zeros=ignore_zeros_plot)

        # Averages
        average_zero = compute_average(in_degrees,ignore_zero=False)
        average_nonzero = compute_average(in_degrees,ignore_zero=True)
        print("The average in-degree (including zeros): " + str(average_zero))
        print("The average in-degree (excluding zeros): " + str(average_nonzero))
        
    def analyze_out_degrees(self,ignore_zeros_plot = True):
        out_degrees = compute_x_degrees(self.prepped_dataset,'out')

        # Distribution
        plot_distribution(out_degrees,title='Log distribution of out-degrees',ignore_zeros=ignore_zeros_plot)

        # Averages
        average_zero = compute_average(out_degrees,ignore_zero=False)
        average_nonzero = compute_average(out_degrees,ignore_zero=True)
        print("The average out-degree (including zeros): " + str(average_zero))
        print("The average out-degree (excluding zeros): " + str(average_nonzero))
        
    def analyze_ram_requirements(self):
        num_links = self.prepped_dataset.get_shape()[0]
        average_links = compute_average_link_count(self.prepped_dataset)

        ram_traditional, ram_traditional_unit = compute_ram_traditional(num_links)
        ram_sparse, ram_sparse_unit = compute_ram_sparse(num_links,avg_links=average_links)
        ram_custom, ram_custom_unit = compute_ram_custom_format(num_links, avg_links=average_links)

        print('Implementing the transition matrix M in the traditional manner...')
        print(f'...and storing both M and the initial vector v in RAM requires {ram_traditional} {ram_traditional_unit} of RAM')
        print('------------------')
        print('Implementing the transition matrix M using a sparse matrix...')
        print(f'...and storing both M and the initial vector v in RAM requires {ram_sparse} {ram_sparse_unit} of RAM')
        print('------------------')
        print('Implementing the transition matrix M using the format specified on slide 17 of the instructions...')
        print(f'...and storing both M and the initial vector v in RAM requires {ram_custom} {ram_custom_unit} of RAM')

### **Program Execution**
This section is concerned with parsing the input arguments and determining the execution flow of the program.
___
The `main` function handles the command line arguments and is responsible for the main flow of the program.

In order to do this the function uses the following parameter:
  * `path` - the location for the link data file [default = `prep_data.npz`]

In [12]:
def main(path = 'prep_data.npz'):
    eda = ExploratoryDataAnalysis()
    eda.load_prepped_data(path)
    
    eda.analyze_dead_ends()
    print('\n')
    eda.analyze_in_degrees()
    print('\n')
    eda.analyze_out_degrees()
    print('\n')
    eda.analyze_ram_requirements()

The following snippet passes the start of the program and the command line arguments to the `main` function.

The following command line argument is expected:
  * `path` - the location of the `prep_data.npz` file

In [13]:
if __name__ == "__main__":
    filepath = sys.argv[1]
    main(path=filepath)

NameError: name 'sys' is not defined

The following snippet triggers the manual execuation of the program

In [ ]:
main()